In [201]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

## predict is_canceled

In [212]:
import pandas as pd
trainX = pd.read_csv('preprocess_train_dummy.csv')
trainY = pd.read_csv('is_canceled.csv')

In [213]:
trainX = trainX.drop('arrival_date', 1)
# trainY = trainY.drop('arrival_date', 1)

In [214]:
# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    trainX, trainY, test_size=0.1, random_state=123
)
y_train

,is_canceled
53000,1
57093,0
37964,0
60638,1
61468,0
...,...
63206,0
61404,1
17730,0
28030,0


In [215]:
from sklearn.metrics import mean_absolute_error
origin_params = {"objective":"binary:logistic", "n_estimators":100, "seed":123}
best_params = {'learning_rate': 0.1, 'n_estimators': 700, 'max_depth': 6, 'min_child_weight': 1, 'seed': 123,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
# best_params = {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 4, 'seed': 123,
# 'subsample': 0.7, 'colsample_bytree': 0.8}
xgbc = xgb.XGBClassifier(**best_params)
xgbc.fit(X_train, y_train)
preds = xgbc.predict(X_train)
mae = mean_absolute_error(y_train,preds)
preds_val = xgbc.predict(X_test)
mae_val = mean_absolute_error(y_test,preds_val)
print("MAE(train): %f" % (mae))
print("MAE(test): %f" % (mae_val))

C:\Users\Police\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Police\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[12:39:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
MAE(train): 0.098948
MAE(test): 0.123990


In [216]:
xgbc.score(X_test, y_test)

0.8760104872187022

In [217]:
preds

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [218]:
test_data_x = pd.read_csv('preprocess_test_dummy.csv')

In [219]:
arrival_date = list(test_data_x['arrival_date'])

In [220]:
test_data_x = test_data_x.drop('arrival_date', 1)

In [221]:
preds_test = xgbc.predict(test_data_x)

In [222]:
preds_test

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [223]:
df = pd.DataFrame(list(zip(arrival_date, preds_test)), columns =['arrival_date', 'is_canceled']) 

In [224]:
df.to_csv(r'test_label_is_canceled_v2.csv', index = False)

## predict adr

In [244]:
trainX = pd.read_csv('preprocess_train_dummy.csv')
trainY = pd.read_csv('adr.csv')

In [245]:
trainX = trainX.drop('arrival_date', 1)
print(len(trainX), len(trainY))

91531 91531


In [258]:
# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    trainX, trainY, test_size=0.1, random_state=123
)

In [259]:
from sklearn.metrics import mean_squared_error
origin_params = {"objective":"binary:logistic", "n_estimators":100, "seed":123}
best_params = {'learning_rate': 0.1, 'n_estimators': 700, 'max_depth': 9, 'min_child_weight': 1, 'seed': 123,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.1, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
# best_params = {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 4, 'seed': 123,
# 'subsample': 0.7, 'colsample_bytree': 0.8}
xgbc = xgb.XGBRegressor(**best_params)
xgbc.fit(X_train, y_train)
preds = xgbc.predict(X_train)
mae = mean_squared_error(y_train,preds)
preds_val = xgbc.predict(X_test)
mae_val = mean_squared_error(y_test,preds_val)
print("MSE(train): %f" % (mae))
print("MSE(test): %f" % (mae_val))

MSE(train): 144.524836
MSE(test): 311.669914


In [260]:
xgbc.score(X_test, y_test)

0.8633648116326276

In [261]:
test_data_x = pd.read_csv('preprocess_test_dummy.csv')

In [262]:
arrival_date = list(test_data_x['arrival_date'])

In [263]:
test_data_x = test_data_x.drop('arrival_date', 1)

In [264]:
preds_test = xgbc.predict(test_data_x)

In [265]:
preds_test

array([ 96.68845 ,  59.287796,  62.663197, ..., 194.22566 ,  86.901665,
        82.893776], dtype=float32)

In [266]:
df = pd.DataFrame(list(zip(arrival_date, preds_test)), columns =['arrival_date', 'adr']) 

In [267]:
df.to_csv(r'test_label_adr_v2.csv', index = False)

## 計算 test daily revenue

In [268]:
test = pd.read_csv('preprocess_test_dummy.csv')
canceled = pd.read_csv('test_label_is_canceled_v2.csv')
adr = pd.read_csv('test_label_adr_v2.csv')

In [269]:
canceled = canceled.drop('arrival_date', 1)
adr = adr.drop('arrival_date', 1)

In [270]:
frames = [test, canceled, adr]

In [271]:
df = pd.concat(frames, axis=1)

In [272]:
columns = df.columns.tolist()

In [273]:
date_list = df['arrival_date'].unique()

In [274]:
n = len(date_list)
n

test_list = []
for i in range(len(df)):
    days = df.loc[i, 'stays_in_weekend_nights'] + df.loc[i,'stays_in_week_nights']
    revenue = df.loc[i, 'adr'] * days
    li = list(df.loc[i,])
    li.append(days)
    li.append(revenue)
    test_list.append(li)

In [275]:
len(test_list)

27859

In [276]:
columns.append('days')
columns.append('revenue')
new_test = pd.DataFrame(test_list, columns=columns)

In [277]:
new_test.to_csv(r'test_predict_all.csv', index = False)

In [278]:
date_list = new_test['arrival_date'].unique()
n = len(date_list)

daily_revenue = []
label = []
for date in date_list:
    df_r = new_test.loc[(new_test['arrival_date'] == date) & (new_test['is_canceled'] == 0)]
    format_date = str(date).split('T')[0]
    N = len(df)

    revenue = df_r["revenue"].sum(axis = 0, skipna = True)
    revenue_scale = int(revenue/10000)
    
    daily_revenue.append(revenue)
    label.append(revenue_scale)

In [279]:
df = {'date': date_list, 'daily_revenue': daily_revenue, 'label': label}

In [280]:
result_df = pd.DataFrame(df)

In [281]:
result_df

,date,daily_revenue,label
0,2017/4/1,31457.182443,3
1,2017/4/2,24037.919621,2
2,2017/4/3,38387.294924,3
3,2017/4/4,13988.659084,1
4,2017/4/5,31460.963054,3
...,...,...,...
148,2017/8/27,60965.909088,6
149,2017/8/28,78823.563062,7
150,2017/8/29,37726.034974,3
151,2017/8/30,30201.154716,3


In [282]:
df = {'arrival_date': date_list, 'label': label}

In [283]:
test_label = pd.DataFrame(df)

In [284]:
test_label.to_csv(r'test_label.csv', index = False)